In [23]:
import yfinance as yf
import pandas as pd
import numpy as np
import vectorbt as vbt
import empyrical as ep
import warnings
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Load data
TFEX_S50 = pd.read_excel("S50_TFEX.xlsx")
TFEX_S50['Timestamp'] = pd.to_datetime(TFEX_S50['Timestamp'])
TFEX_S50.set_index('Timestamp', inplace=True)

# Define the window size
window = 30  # Adjust the window size

# Get the column pairs sequentially
cols = TFEX_S50.columns
num_cols = len(cols)
pairs = [(cols[i], cols[i+1]) for i in range(num_cols - 1)]

for first_col, second_col in pairs:
    print(f"Processing pair: {first_col} and {second_col}")
    
    # Prepare data for the current pair
    P24_H2 = TFEX_S50[[first_col, second_col]].dropna()
    P24_H2[first_col] = pd.to_numeric(P24_H2[first_col], errors='coerce')
    P24_H2[second_col] = pd.to_numeric(P24_H2[second_col], errors='coerce')
    P24_H2['Gap'] = P24_H2[first_col] - P24_H2[second_col]
    P24_H2['Rolling Mean'] = P24_H2['Gap'].rolling(window=window).mean()
    P24_H2['Rolling Std'] = P24_H2['Gap'].rolling(window=window).std()
    P24_H2['Z-score'] = (P24_H2['Gap'] - P24_H2['Rolling Mean']) / P24_H2['Rolling Std']
    P24_H2['EMA'] = P24_H2['Gap'].ewm(span=window, adjust=False).mean()
    
    # Plotting (uncomment if needed)
    # fig = make_subplots(
    #     rows=6, cols=1,
    #     shared_xaxes=True,
    #     subplot_titles=(
    #         f'Price of {first_col}', 
    #         f'Price of {second_col}', 
    #         'Gap', 
    #         'Rolling Mean', 
    #         'EMA', 
    #         'Rolling Std and Z-Score'
    #     ),
    #     vertical_spacing=0.1
    # )

    # fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2[first_col], mode='lines', name=first_col, line=dict(color='blue')), row=1, col=1)
    # fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2[second_col], mode='lines', name=second_col, line=dict(color='orange')), row=2, col=1)
    # fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2['Gap'], mode='lines', name='Gap', line=dict(color='purple')), row=3, col=1)
    # fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2['Rolling Mean'], mode='lines', name='Rolling Mean', line=dict(color='green')), row=4, col=1)
    # fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2['EMA'], mode='lines', name='EMA', line=dict(color='orange', dash='dot')), row=5, col=1)
    # fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2['Rolling Std'], mode='lines', name='Rolling Std', line=dict(color='red')), row=6, col=1)
    # fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2['Z-score'], mode='lines', name='Z-Score', line=dict(color='cyan')), row=6, col=1)

    # fig.update_layout(
    #     title=f'Prices, Gap, Rolling Statistics, EMA, and Z-Score between {first_col} and {second_col}',
    #     xaxis_title='Date',
    #     yaxis_title='Value',
    #     height=1800,
    #     margin=dict(t=100, b=100, l=80, r=80),
    #     template='plotly_dark'
    # )
    # fig.show()
    
    entry_long = P24_H2['Z-score'] < -0.5 * 2
    entry_short = P24_H2['Z-score'] > 0.5 * 2
    exit_long_short = abs(P24_H2['Z-score']) < 0.15

    portfolio = vbt.Portfolio.from_signals(
        close=P24_H2[first_col],
        entries=entry_long,
        short_entries=entry_short,
        exits=exit_long_short,
        short_exits=exit_long_short,
        freq='1D'
    )

    # Output the performance
    print(f"Performance stats for {first_col} and {second_col}:")
    print(portfolio.stats())
    
    fig = portfolio.plot()
    fig.update_layout(title=f'Trading Strategy Performance for {first_col} and {second_col}', xaxis_title='Date', yaxis_title='Portfolio Value')
    fig.show()


Processing pair: S50H0^2 and S50M0^2
Performance stats for S50H0^2 and S50M0^2:
Start                                         NaT
End                           2020-03-30 00:00:00
Period                           63 days 00:00:00
Start Value                                 100.0
End Value                               125.83224
Total Return [%]                         25.83224
Benchmark Return [%]                   -32.068549
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                          6.48393
Max Drawdown Duration             5 days 00:00:00
Total Trades                                    2
Total Closed Trades                             1
Total Open Trades                               1
Open Trade PnL                           1.073899
Win Rate [%]                                100.0
Best Trade [%]                          24.758341
Worst Trade [%]                         24.758341
Avg Winning Trade [%

Processing pair: S50M0^2 and S50U0^2
Performance stats for S50M0^2 and S50U0^2:
Start                                         NaT
End                           2020-06-29 00:00:00
Period                          123 days 00:00:00
Start Value                                 100.0
End Value                              106.759059
Total Return [%]                         6.759059
Benchmark Return [%]                   -18.321359
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        17.658166
Max Drawdown Duration            42 days 00:00:00
Total Trades                                    6
Total Closed Trades                             5
Total Open Trades                               1
Open Trade PnL                         -10.592362
Win Rate [%]                                 60.0
Best Trade [%]                          19.799201
Worst Trade [%]                         -7.821605
Avg Winning Trade [%

Processing pair: S50U0^2 and S50Z0^2
Performance stats for S50U0^2 and S50Z0^2:
Start                                                NaT
End                                  2020-09-29 00:00:00
Period                                 183 days 00:00:00
Start Value                                        100.0
End Value                                     125.261884
Total Return [%]                               25.261884
Benchmark Return [%]                          -25.416667
Max Gross Exposure [%]                             100.0
Total Fees Paid                                      0.0
Max Drawdown [%]                                17.49536
Max Drawdown Duration                   85 days 00:00:00
Total Trades                                          14
Total Closed Trades                                   13
Total Open Trades                                      1
Open Trade PnL                                       0.0
Win Rate [%]                                   53.846154
Best Tra

Processing pair: S50Z0^2 and S50H1^2
Performance stats for S50Z0^2 and S50H1^2:
Start                                         NaT
End                           2020-12-29 00:00:00
Period                          182 days 00:00:00
Start Value                                 100.0
End Value                              111.335625
Total Return [%]                        11.335625
Benchmark Return [%]                    29.178101
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        10.063509
Max Drawdown Duration            75 days 00:00:00
Total Trades                                   13
Total Closed Trades                            12
Total Open Trades                               1
Open Trade PnL                           1.833087
Win Rate [%]                                 50.0
Best Trade [%]                           6.440997
Worst Trade [%]                         -3.277915
Avg Winning Trade [%

Processing pair: S50H1^2 and S50M1^2
Performance stats for S50H1^2 and S50M1^2:
Start                                         NaT
End                           2021-03-30 00:00:00
Period                          183 days 00:00:00
Start Value                                 100.0
End Value                                94.67197
Total Return [%]                         -5.32803
Benchmark Return [%]                    12.157586
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        23.455436
Max Drawdown Duration           111 days 00:00:00
Total Trades                                   18
Total Closed Trades                            17
Total Open Trades                               1
Open Trade PnL                                0.0
Win Rate [%]                            52.941176
Best Trade [%]                           6.794617
Worst Trade [%]                         -9.083098
Avg Winning Trade [%

...

Processing pair: S50U3^2 and S50Z3^2
Performance stats for S50U3^2 and S50Z3^2:
Start                                               NaT
End                                 2023-09-28 00:00:00
Period                                185 days 00:00:00
Start Value                                       100.0
End Value                                    100.124998
Total Return [%]                               0.124998
Benchmark Return [%]                          -8.218599
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               6.821375
Max Drawdown Duration                  68 days 00:00:00
Total Trades                                         16
Total Closed Trades                                  15
Total Open Trades                                     1
Open Trade PnL                                 1.279566
Win Rate [%]                                  46.666667
Best Trade [%]          

Processing pair: S50Z3^2 and S50H4^2
Performance stats for S50Z3^2 and S50H4^2:
Start                                         NaT
End                           2023-12-27 00:00:00
Period                          182 days 00:00:00
Start Value                                 100.0
End Value                               99.477288
Total Return [%]                        -0.522712
Benchmark Return [%]                    -9.141162
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                         9.280351
Max Drawdown Duration            81 days 00:00:00
Total Trades                                   10
Total Closed Trades                             9
Total Open Trades                               1
Open Trade PnL                          -0.204858
Win Rate [%]                            66.666667
Best Trade [%]                           3.232686
Worst Trade [%]                         -6.322884
Avg Winning Trade [%

Processing pair: S50H4^2 and S50M4^2
Performance stats for S50H4^2 and S50M4^2:
Start                                         NaT
End                           2024-03-28 00:00:00
Period                          187 days 00:00:00
Start Value                                 100.0
End Value                              101.671876
Total Return [%]                         1.671876
Benchmark Return [%]                    -7.059476
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        11.994581
Max Drawdown Duration           143 days 00:00:00
Total Trades                                    9
Total Closed Trades                             8
Total Open Trades                               1
Open Trade PnL                           1.349449
Win Rate [%]                                 62.5
Best Trade [%]                           4.474506
Worst Trade [%]                         -9.151013
Avg Winning Trade [%

Processing pair: S50M4^2 and S50U4
Performance stats for S50M4^2 and S50U4:
Start                                         NaT
End                           2024-06-27 00:00:00
Period                          182 days 00:00:00
Start Value                                 100.0
End Value                              106.188195
Total Return [%]                         6.188195
Benchmark Return [%]                   -10.710209
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                         6.498609
Max Drawdown Duration            79 days 00:00:00
Total Trades                                   10
Total Closed Trades                             9
Total Open Trades                               1
Open Trade PnL                           0.281777
Win Rate [%]                            66.666667
Best Trade [%]                           3.967611
Worst Trade [%]                         -3.262606
Avg Winning Trade [%]   

Processing pair: S50U4 and S50Z4
Performance stats for S50U4 and S50Z4:
Start                                         NaT
End                           2024-07-19 00:00:00
Period                          138 days 00:00:00
Start Value                                 100.0
End Value                              100.396829
Total Return [%]                         0.396829
Benchmark Return [%]                    -5.387283
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                         6.498027
Max Drawdown Duration            66 days 00:00:00
Total Trades                                    6
Total Closed Trades                             5
Total Open Trades                               1
Open Trade PnL                          -1.064853
Win Rate [%]                                 80.0
Best Trade [%]                           1.635487
Worst Trade [%]                          -3.12201
Avg Winning Trade [%]       